# TRT KPI: Minimize Time to Science
### IN PROGRESS: Updated in PI 22.2 (Spring 2022) 

The following KPIs are measured in this notebook:

* Number of lines of code for a given workflow in the cloud / Number of lines for same workflow on-prem
* Run time for a given workflow in the cloud / run time for same workflow on-prem

#### Workflows: 
* Basic data search, customize, access, plotting, with a collection that is available on-prem and in Earthdata Cloud.

* Data tested (updated in PI 22.2):
    * Using Cloud Metrics to select popular service combination:
        * Synchronous, L2SS, NetCDF, Spatial & variable request
            * Can we filter by non-admin and by collection requested?
    * Working off of [Openscapes OPeNDAP tutorial](https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/06_S6_OPeNDAP_Access_Gridding.html)

```
ShortName = 'JASON_CS_S6A_L2_ALT_LR_RED_OST_NRT_F'
concept_id = 'C1968980576-POCLOUD'
```
    * On Prem subsetting:
        * [PO.DAAC python script: Subset L1/L2 using OPeNDAP](https://podaac-tools.jpl.nasa.gov/drive/files/common/sw/subsetting/subset_dataset_l2.py)